In [1]:
import os
import sys
import glob

sys.path.append("Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("Hand_pose_estimation_3D")

from dataloader_ann import HandArmLandmarksDataset_ANN

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime

from utilities import fuse_landmarks_from_two_cameras
from functools import partial

import shutil

In [2]:
fusing_model = partial(fuse_landmarks_from_two_cameras,
    tolerance=1e-5,
    method_name="L-BFGS-B")

In [3]:
DATA_DIR = "data"  
SELECTED_DATE = "*"

train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
body_distance_thres=550
leftarm_distance_thres=550
lefthand_distance_thres=200

train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    body_distance_thres, 
    leftarm_distance_thres, 
    lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True)

In [4]:
input_data = train_dataset._inputs

In [5]:
input_data.shape

(10876, 322)

# Slicing data and performing for all data

In [35]:
left_camera_first_intrinsic_value_idx = 144
right_camera_first_lmk_value_idx = left_camera_first_intrinsic_value_idx + 9
right_camera_first_intrinsic_value_idx = right_camera_first_lmk_value_idx + 144
first_right_2_left_matrix_value_idx = right_camera_first_intrinsic_value_idx + 9

left_camera_lmks = input_data[:, :left_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
left_camera_intrinsic = input_data[:, left_camera_first_intrinsic_value_idx:right_camera_first_lmk_value_idx]  # shape: (N, 9), N = #rows
right_camera_lmks = input_data[:, right_camera_first_lmk_value_idx:right_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
right_camera_intrinsic = input_data[:, right_camera_first_intrinsic_value_idx:first_right_2_left_matrix_value_idx]  # shape: (N, 9), N = #rows
right_2_left_mat = input_data[:, first_right_2_left_matrix_value_idx:]  # shape: (N, 16), N = #rows

In [36]:
right_camera_lmks.shape

(10876, 144)

Get records which have depth (z != 0)

In [37]:
left_camera_lmks = left_camera_lmks.reshape(-1, 3, 48)  # shape: (N, 3, 48)
left_camera_lmks_z_values = left_camera_lmks[:, -1, :]
left_camera_rows_contain_z_idx = np.where(np.sum(left_camera_lmks_z_values, axis=1) != 0)[0]
left_camera_rows_contain_z_idx.shape

(4546,)

In [38]:
right_camera_lmks = right_camera_lmks.reshape(-1, 3, 48)
right_camera_lmks_z_values = right_camera_lmks[:, -1, :]
right_camera_rows_contain_z_idx = np.where(np.sum(right_camera_lmks_z_values, axis=1) != 0)[0]
right_camera_rows_contain_z_idx.shape

(4546,)

In [39]:
common_id_mask = left_camera_rows_contain_z_idx == right_camera_rows_contain_z_idx
fake_rows_selected_id = left_camera_rows_contain_z_idx[common_id_mask]

Set depth = 0

In [13]:
fake_left_camera_lmks = left_camera_lmks[fake_rows_selected_id]
fake_left_camera_lmks.shape

(4546, 3, 48)

In [14]:
np.count_nonzero(fake_left_camera_lmks[:, -1, :])

114372

In [15]:
mask = np.random.choice([0, 1], size=(fake_left_camera_lmks[:, -1, :].shape), p=[0.75, 0.25])

In [16]:
fake_left_camera_lmks[:, -1, :] = fake_left_camera_lmks[:, -1, :] * np.random.choice([0, 1], size=(fake_left_camera_lmks[:, -1, :].shape), p=[0.75, 0.25])
#fake_left_camera_lmks[:, -1, :] = np.ones_like(fake_left_camera_lmks[:, -1, :]) 
#fake_left_camera_lmks[:, -1, :] = fake_left_camera_lmks[:, -1, :] * mask

In [17]:
np.count_nonzero(fake_left_camera_lmks[:, -1, :])

28274

In [18]:
fake_right_camera_lmks = right_camera_lmks[fake_rows_selected_id]
fake_right_camera_lmks.shape

(4546, 3, 48)

In [19]:
np.count_nonzero(fake_right_camera_lmks[:, -1, :])

115613

In [20]:
fake_right_camera_lmks[:, -1, :] = fake_right_camera_lmks[:, -1, :] * np.random.choice([0, 1], size=(fake_left_camera_lmks[:, -1, :].shape), p=[0.75, 0.25])
#fake_right_camera_lmks[:, -1, :] = np.ones_like(fake_left_camera_lmks[:, -1, :])
#fake_right_camera_lmks[:, -1, :] = fake_right_camera_lmks[:, -1, :] * mask
np.count_nonzero(fake_right_camera_lmks[:, -1, :])

29087

In [21]:
fake_left_camera_lmks.shape

(4546, 3, 48)

In [22]:
selected_left_camera_intrinsic = left_camera_intrinsic[fake_rows_selected_id]  # shape: (N*, 9), N* = #selected_rows
selected_right_camera_intrinsic = right_camera_intrinsic[fake_rows_selected_id]  # shape: (N*, 9), N* = #selected_rows
selected_right_2_left_mat = right_2_left_mat[fake_rows_selected_id]  # shape: (N*, 9), N* = #selected_rows

In [23]:
img_w = 1920 
img_h = 1080

In [24]:
fake_left_camera_lmks[:, 0, :] = fake_left_camera_lmks[:, 0, :]  * img_w  # shape: (N*, 3, 48), N* = #selected_rows
fake_left_camera_lmks[:, 1, :] = fake_left_camera_lmks[:, 1, :] * img_h  # shape: (N*, 3, 48), N* = #selected_rows
fake_right_camera_lmks[:, 0, :] = fake_right_camera_lmks[:, 0, :] * img_w  # shape: (N*, 3, 48), N* = #selected_rows
fake_right_camera_lmks[:, 1, :] = fake_right_camera_lmks[:, 1, :] * img_h  # shape: (N*, 3, 48), N* = #selected_rows

In [25]:
fake_left_camera_lmks = np.transpose(fake_left_camera_lmks, (0, 2, 1))  # shape: (N*, 48, 3), N* = #selected_rows
fake_right_camera_lmks = np.transpose(fake_right_camera_lmks, (0, 2, 1))  # shape: (N*, 48, 3)

In [26]:
fake_fusing_data = []
for i in range(fake_left_camera_lmks.shape[0]):
    left_xyZ = fake_left_camera_lmks[i]
    right_xyZ = fake_right_camera_lmks[i]
    left_intr = selected_left_camera_intrinsic[i].reshape(3, 3)
    right_intr = selected_right_camera_intrinsic[i].reshape(3, 3)
    right_2_left_mat = selected_right_2_left_mat[i].reshape(4, 4)
    fused_XYZ = fusing_model(left_xyZ,
        right_xyZ,
        right_intr,
        left_intr,
        right_2_left_mat)
    fake_fusing_data.append(fused_XYZ.flatten())

In [27]:
fake_fusing_data = np.array(fake_fusing_data)

In [28]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder", "right hip", 
 "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", 
 "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", 
 "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", 
 "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", 
 "PINKY_DIP", "PINKY_TIP", "right elbow"]

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]

In [29]:
gt_lmks = train_dataset._outputs
gt_lmks = gt_lmks.reshape(-1, 3, 48)
gt_lmks = np.transpose(gt_lmks, (0, 2, 1))

merged_fake_and_gt = np.concatenate([gt_lmks, fake_fusing_data.reshape(-1, 48, 3)], axis=0)

Plot ground-truth and fake data

In [32]:
import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time

time_sleep = 0.1
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)

for i in range(merged_fake_and_gt.shape[0]):
    output = merged_fake_and_gt[i, ...]  # shape: (48, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)

    if i < gt_lmks.shape[0]:
        colors = [[1, 0, 0] for i in range(len(lines))]
    else:
        colors = [[0, 0, 1] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Plot fake data

In [32]:
import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time

time_sleep = 0.1
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)

for i in range(fake_fusing_data.shape[0]):
    output = fake_fusing_data[i, ...]  # shape: (144)
    output = output.reshape(48, 3)  # shape: (48, 3)

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Calculate abs-error between GT data and fake data (depth = 0)

In [30]:
train_dataset._outputs

array([[ 403.9329122 ,  415.46720667,  379.16550393, ..., 1419.42772941,
        1415.19516004, 1415.31684475],
       [ 326.01000858,  343.48034908,  300.78903567, ..., 1546.32257681,
        1544.67232153, 1544.73116729],
       [ 316.26863112,  337.91649653,  282.09465758, ..., 1547.76046339,
        1546.84398216, 1545.51210793],
       ...,
       [ 230.62679044,  379.42922073,  220.95343313, ..., 2766.5307005 ,
        2768.7075366 , 2762.95119337],
       [ 311.68723018,  441.58801223,  278.17032416, ..., 2696.96603027,
        2702.60665757, 2701.79548023],
       [ 419.18539214,  515.68079145,  389.69519384, ..., 2595.4959104 ,
        2605.77249544, 2605.59853492]])

In [31]:
fake_fusing_data

array([[ 403.9329168 , -162.53821442, 1452.78067729, ...,  149.07896002,
         317.40656406, 1415.31684744],
       [ 326.01003582, -178.32397917, 1582.61388768, ...,   76.92987689,
         306.79857738, 1544.73116736],
       [ 316.26863975, -183.05685874, 1542.64847715, ...,   60.09843453,
         311.49169781, 1545.51209429],
       ...,
       [  44.49103033, -125.69358541, 1642.23505562, ...,   -0.        ,
          -0.        ,    0.        ],
       [  45.47561712, -124.03909647, 1642.05288773, ...,   -0.        ,
          -0.        ,    0.        ],
       [  43.72238874, -129.89987106, 1642.08141943, ...,   -0.        ,
          -0.        ,    0.        ]])

In [32]:
true_outputs = train_dataset._outputs
error = true_outputs - fake_fusing_data
abs_error = np.absolute(error)
mean_abs_error = np.mean(abs_error)
print(mean_abs_error)

ValueError: operands could not be broadcast together with shapes (10876,144) (4546,144) 

In [34]:
fake_fusing_data.shape

(4546, 144)

# Perform for each .csv file

In [40]:
from csv_writer import create_csv, append_to_csv, fusion_csv_columns_name

In [41]:
FAKE_DATA_DIR = "data/fake_data"
FAKE_TRAIN_DIR = os.path.join(FAKE_DATA_DIR, "train")
FAKE_VAL_DIR = os.path.join(FAKE_DATA_DIR, "val")

if os.path.exists(FAKE_TRAIN_DIR):
    shutil.rmtree(FAKE_TRAIN_DIR)
os.makedirs(FAKE_TRAIN_DIR)

if os.path.exists(FAKE_VAL_DIR):
    shutil.rmtree(FAKE_VAL_DIR)
os.makedirs(FAKE_VAL_DIR)

In [42]:
seq_len = 5
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
body_distance_thres=550
leftarm_distance_thres=550
lefthand_distance_thres=200

left_camera_first_intrinsic_value_idx = 144
right_camera_first_lmk_value_idx = left_camera_first_intrinsic_value_idx + 9
right_camera_first_intrinsic_value_idx = right_camera_first_lmk_value_idx + 144
first_right_2_left_matrix_value_idx = right_camera_first_intrinsic_value_idx + 9

img_w = 1920 
img_h = 1080

for train_path in train_paths:
    train_dataset = HandArmLandmarksDataset_ANN([train_path], 
        body_lines, 
        lefthand_lines, 
        body_distance_thres, 
        leftarm_distance_thres, 
        lefthand_distance_thres,
        filter_outlier=True,
        only_keep_frames_contain_lefthand=True)
    
    input_data = train_dataset._inputs
    if input_data.shape[0] < seq_len:
        continue
    
    # Slicing data
    left_camera_lmks = input_data[:, :left_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
    left_camera_intrinsic = input_data[:, left_camera_first_intrinsic_value_idx:right_camera_first_lmk_value_idx]  # shape: (N, 9), N = #rows
    right_camera_lmks = input_data[:, right_camera_first_lmk_value_idx:right_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
    right_camera_intrinsic = input_data[:, right_camera_first_intrinsic_value_idx:first_right_2_left_matrix_value_idx]  # shape: (N, 9), N = #rows
    right_2_left_mat = input_data[:, first_right_2_left_matrix_value_idx:]  # shape: (N, 16), N = #rows

    # Get left and right records which have depth (z != 0) 
    left_camera_lmks = left_camera_lmks.reshape(-1, 3, 48)  # shape: (N, 3, 48)
    right_camera_lmks = right_camera_lmks.reshape(-1, 3, 48)  # shape: (N, 3, 48)
    left_camera_lmks_z_values = left_camera_lmks[:, -1, :]  # shape: (N, 48)
    right_camera_lmks_z_values = right_camera_lmks[:, -1, :]  # shape: (N, 48)
    left_camera_sum_z = np.sum(left_camera_lmks_z_values)
    right_camera_sum_z = np.sum(right_camera_lmks_z_values)
    if left_camera_sum_z == 0 or right_camera_sum_z == 0:
        continue

    # Randomly set z = 0
    mask_left = np.random.choice([0, 1], size=(left_camera_lmks_z_values.shape), p=[0.75, 0.25])    
    mask_right = np.random.choice([0, 1], size=(left_camera_lmks_z_values.shape), p=[0.75, 0.25])    
    left_camera_lmks[:, -1, :] = mask_left * left_camera_lmks_z_values  # shape: (N, 3, 48)
    right_camera_lmks[:, -1, :] = mask_right * right_camera_lmks_z_values  # shape: (N, 3, 48)

    left_camera_lmks[:, 0, :] = left_camera_lmks[:, 0, :]  * img_w  # shape: (N, 3, 48)
    left_camera_lmks[:, 1, :] = left_camera_lmks[:, 1, :] * img_h  # shape: (N, 3, 48)
    right_camera_lmks[:, 0, :] = right_camera_lmks[:, 0, :] * img_w  # shape: (N, 3, 48)
    right_camera_lmks[:, 1, :] = right_camera_lmks[:, 1, :] * img_h  # shape: (N, 3, 48)


    left_camera_lmks = np.transpose(left_camera_lmks, (0, 2, 1))  # shape: (N, 48, 3)
    right_camera_lmks = np.transpose(right_camera_lmks, (0, 2, 1))  # shape: (N, 48, 3)

    # Fuse data
    fake_fusing_data = []
    for i in range(left_camera_lmks.shape[0]):
        left_xyZ = left_camera_lmks[i]
        right_xyZ = right_camera_lmks[i]
        left_intr = left_camera_intrinsic[i].reshape(3, 3)
        right_intr = right_camera_intrinsic[i].reshape(3, 3)
        trans_mat = right_2_left_mat[i].reshape(4, 4)
        fused_XYZ = fusing_model(opposite_xyZ=left_xyZ,
            right_side_xyZ=right_xyZ,
            right_side_cam_intrinsic=right_intr,
            opposite_cam_intrinsic=left_intr,
            right_to_opposite_correctmat=trans_mat)
        fake_fusing_data.append(fused_XYZ.flatten())

    fake_fusing_data = np.array(fake_fusing_data)

    filename = os.path.basename(train_path)
    fake_data_path = os.path.join(FAKE_TRAIN_DIR, "fake_" + filename)
    create_csv(fake_data_path, fusion_csv_columns_name)

    # Write to file
    for i in range(fake_fusing_data.shape[0]):
        left_xyZ = left_camera_lmks[i]  # shape: (48, 3)
        right_xyZ = right_camera_lmks[i]  # shape: (48, 3)
        left_intr = left_camera_intrinsic[i]  # shape: (9)
        right_intr = right_camera_intrinsic[i]  # shape: (9)
        trans_mat = right_2_left_mat[i]  # shape: (16)

        fused_lmks = fake_fusing_data[i, ...]  # shape: (144)

        left_xyZ = left_xyZ.T  # shape: (3, 48)
        left_xyZ[0, :] = left_xyZ[0, :] / img_w
        left_xyZ[1, :] = left_xyZ[1, :] / img_h
        right_xyZ = right_xyZ.T  # shape: (3, 48)
        right_xyZ[0, :] = right_xyZ[0, :] / img_w
        right_xyZ[1, :] = right_xyZ[1, :] / img_h
        fused_lmks = fused_lmks.reshape(48, 3)  # shape: (48, 3)
        fused_lmks = fused_lmks.T  # shape: (3, 48)
        
        input_row = np.concatenate([[i],
            left_xyZ.flatten(),
            left_intr.flatten(),
            right_xyZ.flatten(),
            right_intr.flatten(),
            trans_mat.flatten(),
            fused_lmks.flatten()])
        append_to_csv(fake_data_path, input_row)

In [43]:
body_distance_thres = 450
leftarm_distance_thres = 450
lefthand_distance_thres = 200

left_camera_first_intrinsic_value_idx = 144
right_camera_first_lmk_value_idx = left_camera_first_intrinsic_value_idx + 9
right_camera_first_intrinsic_value_idx = right_camera_first_lmk_value_idx + 144
first_right_2_left_matrix_value_idx = right_camera_first_intrinsic_value_idx + 9

img_w = 1920 
img_h = 1080

for val_path in val_paths:
    val_dataset = HandArmLandmarksDataset_ANN([val_path], 
        body_lines, 
        lefthand_lines, 
        body_distance_thres, 
        leftarm_distance_thres, 
        lefthand_distance_thres,
        filter_outlier=True,
        only_keep_frames_contain_lefthand=True)
    
    input_data = val_dataset._inputs
    if input_data.shape[0] < seq_len:
        continue
    
    # Slicing data
    left_camera_lmks = input_data[:, :left_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
    left_camera_intrinsic = input_data[:, left_camera_first_intrinsic_value_idx:right_camera_first_lmk_value_idx]  # shape: (N, 9), N = #rows
    right_camera_lmks = input_data[:, right_camera_first_lmk_value_idx:right_camera_first_intrinsic_value_idx]  # shape: (N, 144), N = #rows
    right_camera_intrinsic = input_data[:, right_camera_first_intrinsic_value_idx:first_right_2_left_matrix_value_idx]  # shape: (N, 9), N = #rows
    right_2_left_mat = input_data[:, first_right_2_left_matrix_value_idx:]  # shape: (N, 16), N = #rows

    # Get left and right records which have depth (z != 0) 
    left_camera_lmks = left_camera_lmks.reshape(-1, 3, 48)  # shape: (N, 3, 48)
    right_camera_lmks = right_camera_lmks.reshape(-1, 3, 48)  # shape: (N, 3, 48)
    left_camera_lmks_z_values = left_camera_lmks[:, -1, :]  # shape: (N, 48)
    right_camera_lmks_z_values = right_camera_lmks[:, -1, :]  # shape: (N, 48)
    left_camera_sum_z = np.sum(left_camera_lmks_z_values)
    right_camera_sum_z = np.sum(right_camera_lmks_z_values)
    if left_camera_sum_z == 0 or right_camera_sum_z == 0:
        continue

    # Randomly set z = 0
    mask_left = np.random.choice([0, 1], size=(left_camera_lmks_z_values.shape), p=[0.75, 0.25])    
    mask_right = np.random.choice([0, 1], size=(left_camera_lmks_z_values.shape), p=[0.75, 0.25])    
    left_camera_lmks[:, -1, :] = mask_left * left_camera_lmks_z_values  # shape: (N, 3, 48)
    right_camera_lmks[:, -1, :] = mask_right * right_camera_lmks_z_values  # shape: (N, 3, 48)

    left_camera_lmks[:, 0, :] = left_camera_lmks[:, 0, :]  * img_w  # shape: (N, 3, 48)
    left_camera_lmks[:, 1, :] = left_camera_lmks[:, 1, :] * img_h  # shape: (N, 3, 48)
    right_camera_lmks[:, 0, :] = right_camera_lmks[:, 0, :] * img_w  # shape: (N, 3, 48)
    right_camera_lmks[:, 1, :] = right_camera_lmks[:, 1, :] * img_h  # shape: (N, 3, 48)


    left_camera_lmks = np.transpose(left_camera_lmks, (0, 2, 1))  # shape: (N, 48, 3)
    right_camera_lmks = np.transpose(right_camera_lmks, (0, 2, 1))  # shape: (N, 48, 3)

    # Fuse data
    fake_fusing_data = []
    for i in range(left_camera_lmks.shape[0]):
        left_xyZ = left_camera_lmks[i]
        right_xyZ = right_camera_lmks[i]
        left_intr = left_camera_intrinsic[i].reshape(3, 3)
        right_intr = right_camera_intrinsic[i].reshape(3, 3)
        trans_mat = right_2_left_mat[i].reshape(4, 4)
        fused_XYZ = fusing_model(opposite_xyZ=left_xyZ,
            right_side_xyZ=right_xyZ,
            right_side_cam_intrinsic=right_intr,
            opposite_cam_intrinsic=left_intr,
            right_to_opposite_correctmat=trans_mat)
        fake_fusing_data.append(fused_XYZ.flatten())

    fake_fusing_data = np.array(fake_fusing_data)

    filename = os.path.basename(val_path)
    fake_data_path = os.path.join(FAKE_VAL_DIR, "fake_" + filename)
    create_csv(fake_data_path, fusion_csv_columns_name)

    # Write to file
    for i in range(fake_fusing_data.shape[0]):
        left_xyZ = left_camera_lmks[i]  # shape: (48, 3)
        right_xyZ = right_camera_lmks[i]  # shape: (48, 3)
        left_intr = left_camera_intrinsic[i]  # shape: (9)
        right_intr = right_camera_intrinsic[i]  # shape: (9)
        trans_mat = right_2_left_mat[i]  # shape: (16)

        fused_lmks = fake_fusing_data[i, ...]  # shape: (144)

        left_xyZ = left_xyZ.T  # shape: (3, 48)
        left_xyZ[0, :] = left_xyZ[0, :] / img_w
        left_xyZ[1, :] = left_xyZ[1, :] / img_h
        right_xyZ = right_xyZ.T  # shape: (3, 48)
        right_xyZ[0, :] = right_xyZ[0, :] / img_w
        right_xyZ[1, :] = right_xyZ[1, :] / img_h
        fused_lmks = fused_lmks.reshape(48, 3)  # shape: (48, 3)
        fused_lmks = fused_lmks.T  # shape: (3, 48)
        
        input_row = np.concatenate([[i],
            left_xyZ.flatten(),
            left_intr.flatten(),
            right_xyZ.flatten(),
            right_intr.flatten(),
            trans_mat.flatten(),
            fused_lmks.flatten()])
        append_to_csv(fake_data_path, input_row)